In [1]:
import os
import shutil
import glob
from obspy import UTCDateTime

files=glob.glob(r'.\*.txt')

In [2]:
class event:
    def __init__(self, lat, lon, depth, t, M):
        self.lat = lat
        self.lon = lon
        self.depth = depth
        self.t = t
        self.M = M

In [3]:
output = []
fnum = 0
for file in files:
    f = open(file)
    lines = f.readlines()
    isStart = 'Y'
    evn_n = 0
    evn_ls = []
    del lines[0]
    del lines[0]
    for line in lines:
        #print(line)
        if (line=='\n'):
            # finish the last data segment
                evn_ls.append(event(lat,lon,depth,t, M))
                evn_ls[evn_n-1].phs = phs
                evn_ls[evn_n-1].phs_num = len(phs)
                evn_ls[evn_n-1].gap_azi = gap_azi
                break
        if line[0:3]=='DBO':
            if isStart =='Y':
                isStart ='N'
            else:
                # finish last data segment
                evn_ls.append(event(lat,lon,depth,t, M))
                evn_ls[evn_n-1].phs = phs
                evn_ls[evn_n-1].phs_num = len(phs)
                evn_ls[evn_n-1].gap_azi = gap_azi
            #record new data segment
            t = UTCDateTime(line[7:29])
            lat = line[31:37]
            lon = line[39:46]
            depth = line[47:50]
            if (depth.isspace()):
                depth='999'
            phs = []
            evn_n += 1
        if line[0:3]=='DEO':
            M = line[50:54]
            gap_azi = line[72:77]
            if gap_azi.isspace():
                gap_azi = "%7.1f"%float(360)
            else:
                gap_azi = gap_azi.strip()
        if line[0:3]=='DPB':
            #print(line)
            ph_nm = (line[25:28]).upper()
            if (ph_nm!=' PG') and (ph_nm!=' SG') and (ph_nm!=' PN') and (ph_nm!=' SN')\
            and (ph_nm!=' PB')and (ph_nm!=' SB'):
                continue
            ph_t = UTCDateTime(line[34:56])
            st_nm = line[7:12]
            net_nm = line[4:6]
            netst_nm = net_nm.strip()+'.'+st_nm.strip()
            travel_t = ph_t - t
            weight = line[29]
            distance = line[65:71]
            azi = line[72:77]
            if distance.isspace() or weight.isspace():
                continue
            tmp_ph = netst_nm.strip().ljust(7) + "%7.2f"%(travel_t)+"%6.1f"%int(weight)\
            + ph_nm.rjust(5)+"%7.1f"%float(distance)+ ph_t.strftime(' %H:%M:%S.%f')[:-4]\
            +'  0.00'+ azi.rjust(6)
            phs.append(tmp_ph)
            #break
    evn_ls.reverse()
    #事件编号及序号
    pre_mm = 'None'
    sn = 0
    for evn in evn_ls:
        mm = evn.t.strftime('%m')
        if mm!=pre_mm:
            c = 1
            pre_mm = mm
        else:
            c +=1
        evn.code = evn.t.strftime('%y%m')+"%04d"%(c)
        sn += 1
        evn.serial_num = 0 + len(output) + sn
    f.close()
    output = output + evn_ls
    fnum +=1
#     if evn_n>10:
#         break
    #break

In [4]:
ol = []
fo = open('.\\out.dat', 'w+')
for e in output:
    nms = []
    stn = 0
    for ph in e.phs:
        st_nm = ph[0:7]
        if st_nm not in nms:
            nms.append(st_nm)
            stn +=1
    ol.append(e.t.strftime('# %Y %m %d %H %M %S.%f')[:-4]+"%9.4f"%float(e.lat)+"%9.4f"%float(e.lon)\
             +"%6.1f"%float(e.depth)+e.M.rjust(5)+'  0.0  0.0   0.000'+"%7d"%(e.serial_num)\
              +"%5d"%(e.phs_num)+"%5d"%(stn)+'    0.0'+e.gap_azi.rjust(7))
#     for ph in e.phs:
#         if ph[50:].isspace():
#             tmp_ph = list(ph)
#             tmp_ph[50:] = e.phs[-1][50:]
#             ph = ''.join(tmp_ph)
    for i in range(len(e.phs)):
        ph = e.phs[i]
        if ph[50:].isspace():
            for j in range(len(e.phs)):
                ph2 = e.phs[j]
                if (ph[0:7]==ph2[0:7]) and ((ph2[20:25].strip()=='PG')or(ph2[20:25].strip()=='SG')) and not ph2[50:].isspace():
                    tmp_ph = list(ph)
                    tmp_ph[50:] = ph2[50:]
                    ph = ''.join(tmp_ph)
            if ph[50:].isspace():
                tmp_ph = list(ph)
                tmp_ph[50:] = "%6.1f"%float(360)
                ph = ''.join(tmp_ph)
        ol.append(ph)
for l in ol:
    fo.write(l+'\n')
fo.close()

for i in range(len(e.phs)):
    ph = e.phs[i]
    if ph[50:].isspace():
        for j in range(len(e.phs)):
            ph2 = e.phs[j]
            if (ph[0:7]==ph2[0:7]) and ((ph2[20:25].strip()=='PG')or(ph2[20:25].strip()=='SG')) and not ph2[50:].isspace():
                tmp_ph = list(ph)
                tmp_ph[50:] = ph2[50:]
                ph = ''.join(tmp_ph)
        if ph[50:].isspace():
            tmp_ph = list(ph)
            tmp_ph[50:] = "%6.1f"%float(360)
            ph = ''.join(tmp_ph)

In [5]:
not '  '.isspace()

False

In [6]:
ph[0:7]

'GD.NAP '

201307011206